In [1]:
import os, sys, time
import numpy as np
import pandas as pd
from tqdm import tqdm
import cPickle as pk
np.random.seed(1) # to be reproductive

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import Conv2D, MaxPooling2D, Reshape
from keras.models import Sequential
from keras.models import load_model
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from keras.callbacks import Callback, EarlyStopping
import keras.backend as K
from keras.layers import Input, Dense, Activation
from keras.models import Model

Using TensorFlow backend.


In [2]:
ENCODING_DIM = 500
INPUT_DIM = 59790
NOISE_PORTION = 0.5 # randomly mask protion
VALIDATION_SPLIT = 0.2
BATCH_SZ = 128
NB_EPOCH = 50

In [3]:
input_raw = Input(shape=(INPUT_DIM,))
# *** for final dpvec, should I take the activations before relu ??? ***
hiddenlayer = Dense(ENCODING_DIM, activation='relu')
outputlayer = Dense(INPUT_DIM, activation='sigmoid')

encoded = hiddenlayer(input_raw)
decoded = outputlayer(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_raw, output=decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [4]:
X = pk.load(open('./data/umls_raw_feature.pk'))

In [5]:
# X = X[:100]

In [6]:
shuffle_index = np.arange(np.shape(X)[0])
np.random.shuffle(shuffle_index)
X = X[shuffle_index]

In [7]:
train_sz = int(X.shape[0]*(1-VALIDATION_SPLIT))
X_train = X[:train_sz]
X_val = X[train_sz:]
print X_train.shape, X_val.shape

(36916, 59790) (9230, 59790)


In [8]:
def add_noise(X):
    nb_masked = int(INPUT_DIM*NOISE_PORTION)
    masks = []
    mask = [0]*nb_masked+[1]*(INPUT_DIM-nb_masked)
    for i in xrange(X.shape[0]):
        np.random.shuffle(mask)
        masks.append(mask)
    masks = np.array(masks)
    X_noisy = X * masks
    return X_noisy

In [9]:
def batch_generator(X, batch_size=BATCH_SZ): 
    shuffle_index = np.arange(X.shape[0])
    np.random.shuffle(shuffle_index)
    X =  X[shuffle_index, :]
    
    samples_per_epoch = X.shape[0]
    number_of_batches = samples_per_epoch//batch_size
    counter=0
    while 1:
        index_batch = shuffle_index[batch_size*counter: min(samples_per_epoch, batch_size*(counter+1))]
        X_batch = np.array(X[index_batch,:].todense()) # or directly .toarray()... 
        X_batch_noisy = add_noise(X_batch)
        counter += 1
        if (counter >= number_of_batches):
            np.random.shuffle(shuffle_index)
            counter=0
        yield ( X_batch_noisy, X_batch ) # X: corrupted, y: original 

In [10]:
logdir = 'logs/autoencoder'
_callbacks = [EarlyStopping(monitor='val_loss', patience=2),
              TensorBoard(log_dir=logdir, histogram_freq=0, write_graph=False) # 
             ]
# ~11G memory usage
autoencoder.fit_generator(
    generator=batch_generator(X_train, batch_size=BATCH_SZ),
    samples_per_epoch=X_train.shape[0],
    validation_data = batch_generator(X_val, batch_size=BATCH_SZ),
    nb_val_samples = X_val.shape[0],
    nb_epoch=NB_EPOCH, 
    callbacks = _callbacks )
print 'run "tensorboard --logdir=%s" to launch tensorboard'%logdir

Epoch 1/50
36864/36916 [============================>.] - ETA: 0s - loss: 0.6508

/local/XW/SOFT/anaconda2/envs/thesis_nb/lib/python2.7/site-packages/keras/engine/training.py:1462: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


36992/36916 [==============================] - 672s - loss: 0.6500 - val_loss: 0.4213
Epoch 2/50
36992/36916 [==============================] - 672s - loss: 0.1797 - val_loss: 0.1023
Epoch 3/50
36992/36916 [==============================] - 710s - loss: 0.0873 - val_loss: 0.0775
Epoch 4/50
36992/36916 [==============================] - 753s - loss: 0.0730 - val_loss: 0.0700
Epoch 5/50
36992/36916 [==============================] - 784s - loss: 0.0669 - val_loss: 0.0653
Epoch 6/50
36992/36916 [==============================] - 806s - loss: 0.0630 - val_loss: 0.0614
Epoch 7/50
36992/36916 [==============================] - 786s - loss: 0.0601 - val_loss: 0.0601
Epoch 8/50
36992/36916 [==============================] - 704s - loss: 0.0585 - val_loss: 0.0580
Epoch 9/50
36992/36916 [==============================] - 678s - loss: 0.0568 - val_loss: 0.0565
Epoch 10/50
36992/36916 [==============================] - 674s - loss: 0.0557 - val_loss: 0.0555
Epoch 11/50
36992/36916 [===============

# generating deep patient vectors

In [11]:
sid2dpvec = {} # maps sid(int) to the deep patient vector (dim=500, dense vector)

In [12]:
MODEL_PATH = './models/'
model_fpath = os.path.join( MODEL_PATH, 'autoencoder.h5' )
autoencoder.save(model_fpath)

In [16]:
Xtest = X[:10]
Xtest = Xtest.toarray()

In [17]:
Xtest

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [18]:
encoded2 = hiddenlayer(decoded)
decode2 =outputlayer(encoded2)
encoded3 = hiddenlayer(decode2)

In [19]:
dpencoder = Model(input=input_raw, output=encoded3)

In [25]:
dpencoder.save(os.path.join(MODEL_PATH, 'dpencoder.h5'))

In [21]:
dpvec = dpencoder.predict(Xtest) # deep patient feature vecs

In [22]:
dpvec

array([[  0.05022343,   0.91315228,   2.74883175, ...,  10.09865475,
          1.0744679 ,   0.        ],
       [  0.0700089 ,   0.64521551,   3.30494046, ...,   7.43990088,
          1.53842115,   0.        ],
       [  0.07036356,   0.68368161,   3.3699851 , ...,   7.93069601,
          1.55791414,   0.        ],
       ..., 
       [  0.01348547,   1.03034902,   1.77273345, ...,  10.32262516,
          0.46413344,   0.        ],
       [  0.04973714,   0.49226749,   2.75314713, ...,   5.57210684,
          1.22105801,   0.        ],
       [  0.06937402,   0.64268863,   3.3057766 , ...,   7.40880632,
          1.53373861,   0.        ]], dtype=float32)

In [23]:
encoder = Model(input=input_raw, output=encoded) # first hidden layer
encoder.predict(Xtest)

array([[ 0.        ,  1.29652154,  0.97244507, ...,  9.34387779,
         0.        ,  0.        ],
       [ 0.05863165,  0.79796082,  1.24403846, ...,  8.36209679,
         0.54034084,  0.        ],
       [ 0.07767202,  0.78847355,  1.2612294 , ...,  7.23636293,
         0.6363281 ,  0.05814032],
       ..., 
       [ 0.03601322,  0.46379206,  0.85510164, ...,  9.61740685,
         0.04049516,  0.        ],
       [ 0.03429135,  0.63824004,  2.17142415, ...,  5.81032801,
         1.15960836,  0.        ],
       [ 0.        ,  0.77705055,  1.85124552, ...,  8.17969513,
         0.19800772,  0.        ]], dtype=float32)

In [24]:
encoder2 = Model(input=input_raw, output=encoded2) # 2nd hidden layer
encoder2.predict(Xtest)

array([[  9.18666087e-03,   1.03058386e+00,   1.76229382e+00, ...,
          1.02850342e+01,   4.50387120e-01,   0.00000000e+00],
       [  6.03783354e-02,   8.17490637e-01,   3.08688331e+00, ...,
          9.35246086e+00,   1.34000301e+00,   0.00000000e+00],
       [  5.16502447e-02,   8.35695267e-01,   2.89522815e+00, ...,
          9.37288857e+00,   1.21300662e+00,   0.00000000e+00],
       ..., 
       [  0.00000000e+00,   1.07883680e+00,   1.11776173e+00, ...,
          1.02465630e+01,   8.53298604e-02,   0.00000000e+00],
       [  5.39490208e-02,   5.56160033e-01,   2.89145684e+00, ...,
          6.33792496e+00,   1.28273761e+00,   0.00000000e+00],
       [  5.12225330e-02,   8.30166876e-01,   3.15035653e+00, ...,
          9.40587044e+00,   1.30656862e+00,   0.00000000e+00]], dtype=float32)

In [29]:
dpvecs = []
for i in tqdm(xrange(0, X.shape[0], BATCH_SZ)):
    
    x_batch = X[i:min(i+BATCH_SZ, X.shape[0])].toarray()
    dpveci = dpencoder.predict(x_batch)
    dpvecs.append(dpveci)
dpvecs = np.vstack(dpvecs)

100%|██████████| 361/361 [05:35<00:00,  1.30it/s]


In [30]:
dpvecs.shape

(46146, 500)